## REFORMAT DATASET
Reformatting the layout of the overall dataset in terms of column information, tupled statsitical data, and identifying the null values.

In [4]:
##Importing Libraries For Exploration
import pandas as pd

EnvImpact_Data = pd.read_csv("./CorporateEnvImpact_Dataset.csv")
EnvImpact_Data.head(5)

,Year,Company Name,Country,Industry (Exiobase),Total Environmental Intensity (Revenue),Total Environmental Intensity (Operating Income),Total Environmental Cost,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity,% Imputed
0,2018,TELEPERFORMANCE SE,France,Activities auxiliary to financial intermediati...,-1.09%,-10.05%,"(5,52,32,974)","(4,85,90,497)","(11,456)","(6,46,758)","(1,51,520)","(2,061)","(3,661)","(58,28,063)","1,042",2%
1,2018,SGS SA,Switzerland,Activities auxiliary to financial intermediati...,-0.81%,-5.31%,"(5,51,43,250)","(5,07,27,341)","(11,763)","(6,73,791)","(1,57,783)","(2,131)","(3,489)","(35,67,524)",571,0%
2,2018,INTERTEK GROUP PLC,United Kingdom,Activities auxiliary to financial intermediati...,-1.53%,-9.38%,"(5,46,77,862)","(5,34,89,006)","(12,428)","(7,11,263)","(1,66,744)","(2,244)","(3,428)","(2,93,207)",458,1%
3,2018,APPLUS SERVICES SA,Spain,Activities auxiliary to financial intermediati...,-2.26%,-35.02%,"(4,32,98,590)","(1,89,12,678)","(4,652)","(2,49,594)","(58,535)",(811),"(3,850)","(2,40,69,048)",578,2%
4,2018,BUREAU VERITAS SA,France,Activities auxiliary to financial intermediati...,-0.56%,-4.40%,"(3,08,62,191)","(3,01,89,038)","(7,276)","(4,02,067)","(94,143)","(1,298)","(3,953)","(1,65,542)","1,126",3%
